 Önce herhangi bir özel şekilde yapılanmamış olan düz metin dosyalarını işlemeyi göreceğiz. Ardından _csv_ modülüyle CSV biçiminde yapılanmış dosyaları okuyup yazmayı inceleyeceğiz. JSON, ZIP, PDF, Word, Excel, HTML dosyalarının işlenmesine kısaca değineceğiz. Son olarak, Python oturumunda yarattığımız nesneleri ikili (binary) formda dosyaya kaydetmemizi ve sonra dosyadan tekrar yüklememizi sağlayan **`pickle`** modülünün kullanımını özetleyeceğiz.


Genel dosya okuma/yazma
===
Dosya okumak
---
Öncelikle, IPython sihirleriyle _deneme.txt_ isimli bir dosya yaratalım (Jupyter kullanmıyorsanız aşağıdaki metni bir metin editörüne kopyalayıp _deneme.txt_ ismiyle çalışma dizininize kaydedebilirsiniz).

In [1]:
%%writefile deneme.txt
Ey Türk Gençliği!
Birinci vazifen, 
Türk istiklâlini, Türk Cumhuriyeti'ni, 
ilelebet muhafaza ve müdafaa etmektir.

Writing deneme.txt


Bir dosyayı açmak için `open()` fonksiyonunu kullanırız. En basit halinde `open()` mevcut bir metin dosyasını okumak için açar ve bir dosya nesnesi döndürür.

In [2]:
f = open("deneme.txt")

Dosya nesnesinin `read()` metodu dosya içeriğini tek bir dize halinde döndürür.

In [3]:
f.read()

"Ey Türk Gençliği!\nBirinci vazifen, \nTürk istiklâlini, Türk Cumhuriyeti'ni, \nilelebet muhafaza ve müdafaa etmektir."

Açılmış dosyalarla işimiz bittiğinde kapatmamız gerekir, yoksa bellekte birikip yer işgal edebilirler.

In [4]:
f.close()

`readlines()` metodu, satırlardan oluşan bir liste döndürür:

In [5]:
f = open("deneme.txt")
f.readlines()

['Ey Türk Gençliği!\n',
 'Birinci vazifen, \n',
 "Türk istiklâlini, Türk Cumhuriyeti'ni, \n",
 'ilelebet muhafaza ve müdafaa etmektir.']

In [6]:
f.close()

Bir dosya nesnesi bir _iteratördür_; bütün dosyayı bir kerede belleğe yüklemez, ama talep geldikçe satırları birer birer verir. `readline()` metodu dosyadaki mevcut satırı okumak için kullanılır. Her yeni `readline()` çağrısı bir sonraki satırı getirir.

In [7]:
f = open("deneme.txt")
f.readline()

'Ey Türk Gençliği!\n'

In [8]:
f.readline()

'Birinci vazifen, \n'

Dosya nesnesi bir iteratör olduğu için `for` döngüsünde kullanılabilir. Bir dosyayı satır satır işlemek için şöyle bir döngü kurulur.

In [9]:
for satır in f:
    print(satır.upper(),end="")
f.close()

TÜRK ISTIKLÂLINI, TÜRK CUMHURIYETI'NI, 
ILELEBET MUHAFAZA VE MÜDAFAA ETMEKTIR.

with ... as
---
Yukarıda, dosyayı `open()` ile açtıktan sonra `close()` metoduyla kapatmak gerektiğini söyledik. Bu basit bir kural olsa da, karmaşık programlar içinde gözden kaçabiliyor. Dosya açma/kapama işlemini daha düzenli hale getirmek için Python programcıları _context manager_ denen bir yapı kullanırlar. Bir context manager oluşturmak için `with` komutu kullanılır.

In [10]:
with open("deneme.txt") as f:
    print(f.read())

Ey Türk Gençliği!
Birinci vazifen, 
Türk istiklâlini, Türk Cumhuriyeti'ni, 
ilelebet muhafaza ve müdafaa etmektir.


Context manager ayrı ve geniş bir konudur, ayrıntısı için [Python belgelerine](https://docs.python.org/3/reference/compound_stmts.html#with) bakabilirsiniz. Bizim açımızdan önemli olan, burada `with` bloku bitince dosyanın otomatik olarak kapatılmasıdır. Dosyanın kapanmış olduğunu, dosya nesnesinin `closed` değişkeninin durumuyla kontrol edebiliriz.

In [11]:
f.closed

True

Dosyaya yazmak
-----
Bir dosyaya yazmak için `open()` fonksiyonunda `"w"` (write) modunu kullanırız.

In [12]:
f = open("deneme2.txt", "w")

Bu komutla, mevcut dizinde _deneme2.txt_ dosyası yoksa yaratılır, varsa mevcut içeriği silinip üstüne yeni veri yazılır. Açılan dosyaya bir dize yazmak için `write()` metodu kullanılır.

In [13]:
f.write("ABCDE\n")
f.write("123456\n")
f.write("wxyz\n")
f.close()

Dosyayı açarak, veya `%cat` sihirini kullanarak içeriğine bakabiliriz.

In [14]:
%cat deneme2.txt

ABCDE
123456
wxyz


Aynısını bir context manager ile de yapabiliriz:

In [15]:
with open("deneme2.txt", "w") as f:
    f.write("ABCDE\n")
    f.write("123456\n")
    f.write("wxyz\n")

Dosyada mevcut bulunan verileri silmeden, yeni verilerin dosyanın altına eklenmesini istersek dosyayı `"a"` (append) modunda açmalıyız.

In [16]:
with open("deneme2.txt", "a") as f:
    f.write("Yeni satır 1\n")
    f.write("Yeni satır 2\n")

In [17]:
%cat deneme2.txt

ABCDE
123456
wxyz
Yeni satır 1
Yeni satır 2


CSV dosya biçimi
=====
 
Bir çok veri dosyasında veriler tablo halinde, her satırda bir _kayıt_ ve her sütunda o kayda ait bir _alan_ olacak şekilde düzenlenmişlerdir. Alanlar birbirlerinden boşlukla, virgülle, veya başka bir karakterle ayrılabilir. Bu tür bir dosya düzenine CSV (comma-separated values, virgülle ayrılmış değerler) adı verilir. CSV biçimindeki dosyaları okumak için yukarıda açıkladığımız genel yöntemleri kullanmak mümkünse de, Python'un `csv` modülü işleri basitleştirir. Sözgelişi MS Excel ve benzeri bir hesap tablosunu CSV biçiminde kaydedip, verileri Python ile okuyabilirsiniz; veya Python'la üretilen verileri CSV dosyası olarak kaydedip hesap tablosu uygulamasıyla açabilirsiniz.

CSV okuma
-----
Önce bir örnek veri dosyası hazırlayalım.

In [18]:
%%writefile ornek.csv
"Potter, H",37,"Londra, İngiltere"
"Granger, H",36,"Sydney, Avustralya"
"Weasley, Bill",45,"Bükreş, Romanya"

Overwriting ornek.csv


Bu örnekte isim ve adres alanı değerlerinin tırnak içinde yazıldığına dikkat edin. Eğer tırnak kullanılmasaydı, alanları virgülle ayırma kuralı bize `"Potter"`, `"H"`, `37`, `"Londra"`, `"İngiltere"` gibi beş ayrı alan verirdi. 

Bir CSV dosyasını açtıktan sonra, `csv` modülündeki `reader()` fonksiyonunu kullanarak onu satır satır okuyacak bir iteratör nesnesi yaratırız. Sonra bu iteratör nesnesi üzerinde bir döngüyle dosyayı tarayabiliriz.

In [19]:
import csv
with open("ornek.csv") as f:
    okur = csv.reader(f)
    for satır in okur:
        print(satır)

['Potter, H', '37', 'Londra, İngiltere']
['Granger, H', '36', 'Sydney, Avustralya']
['Weasley, Bill', '45', 'Bükreş, Romanya']


Veya dosyamızda alan ayırıcı olarak boşluk karakteri, alan gruplama için bölü işareti (`/`) kullanılmış olabilir. Bu durumda `reader()` fonksiyonundaki `delimiter` ve `quotechar` parametrelerini değiştirerek dosyayı doğru şekilde alabiliriz.

In [20]:
%%writefile ornek2.csv
/Potter, H/ 37 /Londra, İngiltere/
/Granger, H/ 36 /Sydney, Avustralya/
/Weasley, Bill/ 45 /Bükreş, Romanya/

Overwriting ornek2.csv


In [21]:
with open("ornek2.csv") as f:
    okur = csv.reader(f, delimiter=" ", quotechar="/")
    for satır in okur:
        print(satır)

['Potter, H', '37', 'Londra, İngiltere']
['Granger, H', '36', 'Sydney, Avustralya']
['Weasley, Bill', '45', 'Bükreş, Romanya']


Görüldüğü gibi, bir CSV dosyasında alan ayırma, gruplama, satır sonu karakterleri için farklı tercihler olabilir. Her bir tercih kümesine bir _lehçe_ (dialect) adı veriliyor. `csv.reader()` için varsayılan lehçe olan `"excel"`, MS Excel ile üretilen CSV dosyalarını okumaya ayarlıdır. Ancak yukarıda gördüğümüz gibi bu tercihler kolaylıkla değiştirilebilir.

Eğer özel bir CSV biçimini sık sık kullanıyorsanız, o lehçeye özel bir `Dialect` nesnesi oluşturup `reader()` ile birlikte kullanabilirsiniz. Bunun ayrıntılarını [Python belgelerinden](https://docs.python.org/3/library/csv.html) öğrenebilirsiniz.

CSV yazma
---
Elimizdeki verileri bir CSV dosyasına yazmak için önce `csv` modülünün `writer()` fonksiyonuyla bir yazıcı nesnesi yaratırız. Yazıcı nesnesi verilen veriyi kullanılan "lehçe"ye uygun şekilde bir dizeye dönüştürür ve dosyaya yazar. Yazılacak dosyayı `open()` ile açarken `newline=""` parametresi vermemiz gerekir.

Yazıcı nesnesinin `writerow()` metodu yazılacak satırı bir liste olarak alır.

In [22]:
with open("ornek3.csv", "w", newline="") as f:
    yazıcı = csv.writer(f)
    yazıcı.writerow(['Potter, H', '37', 'Londra, İngiltere'])
    yazıcı.writerow(['Granger, H', '36', 'Sydney, Avustralya'])

Dosyanın içeriğine bakarak doğru yazılıp yazılmadığını kontrol edelim:

In [23]:
%cat ornek3.csv

"Potter, H",37,"Londra, İngiltere"
"Granger, H",36,"Sydney, Avustralya"


Pandas ile CSV okuma
---

CSV dosyalarını veri analizi paketi _pandas_ ile de okumak ve yazmak mümkündür. Pandas ile CSV okumak hem daha basittir, hem de `csv` modülünde bulunmayan sözgelişi veri içindeki yorumları elemek, sadece istenen sütunları almak gibi ince ayarlara da imkan verir.

Pandas'ın `read_csv()` fonksiyonu dosyanın içeriğini bir veri çerçevesi olarak okur; ardından bu veri çerçevesinden çeşitli satırlar veya sütunlar alınabilir. Pandas kullanımı bu notların kapsamı dışında olduğu için ayrıntıya girmiyorum.

Diğer dosya formatları
====
Birçok dosya biçimi için Python'da özelleşmiş kütüphaneler vardır. Bunların bazılarına değinelim.
JSON
----
Birçok internet hizmeti API'si, sorgulama sonuçlarını JSON biçiminde bir dosya olarak verir (sözgelişi daha önce [Foursquare](http://www.veridefteri.com/2018/01/23/foursquare-api/), [Weather Underground](http://www.veridefteri.com/2018/03/22/weather-underground-api/) ve [Star Wars](http://www.veridefteri.com/2017/12/13/veri-defteri-ben-seviyor-cok/) API'lerini JSON verilerini incelemiştik). Python standart kütüphanesindeki `json` modülü, JSON biçimli bir dosyayı okuyup bir Python sözlük veya listesine dönüştüren, ve Python nesnelerinden JSON biçimli bir dosya oluşturan fonksiyonları içerir.

[Mockaroo](https://www.mockaroo.com/) sitesini kullanarak yalancı veri içeren küçük bir JSON dosyası üretelim.

In [24]:
%%writefile yalanci_veri.json
[{
  "id": 1,
  "first_name": "Fredia",
  "last_name": "Waith",
  "email": "fwaith0@tamu.edu"
}, {
  "id": 2,
  "first_name": "Rafaello",
  "last_name": "Rowthorn",
  "email": "rrowthorn1@stanford.edu"
}, {
  "id": 3,
  "first_name": "Harriette",
  "last_name": "Patters",
  "email": "hpatters2@samsung.com"
}]

Overwriting yalanci_veri.json


Şimdi bu dosyayı açıp, içeriğini yorumlayalım ve bir Python nesnesine aktaralım.

In [25]:
import json
with open("yalanci_veri.json") as f:
    yalanciveriler = json.load(f)

Bu işlem sonucunda bir sözlükler listesi elde etmiş oluruz. Bilindik indeksleme işlemleriyle tek tek elemanlara ulaşabiliriz.

In [26]:
yalanciveriler[1]["first_name"], yalanciveriler[1]["email"]

('Rafaello', 'rrowthorn1@stanford.edu')

Şimdi JSON dosyası yazmayı görelim. Elimizdeki veriye bir satır daha ekleyelim ve nesnenin yeni halini `json.dump()` ile dosyaya yazalım.

In [27]:
yalanciveriler.append(
{
  "id": 4,
  "first_name": "Giffer",
  "last_name": "Dur",
  "email": "gdur2@cnbc.com"
})

with open("yalanci_veri.json", "w") as f:
    json.dump(yalanciveriler,f)

Dosyanın içeriğine bakarak beklediğimiz şekilde yazıldığını görebiliyoruz.

In [28]:
%cat yalanci_veri.json

[{"id": 1, "first_name": "Fredia", "last_name": "Waith", "email": "fwaith0@tamu.edu"}, {"id": 2, "first_name": "Rafaello", "last_name": "Rowthorn", "email": "rrowthorn1@stanford.edu"}, {"id": 3, "first_name": "Harriette", "last_name": "Patters", "email": "hpatters2@samsung.com"}, {"id": 4, "first_name": "Giffer", "last_name": "Dur", "email": "gdur2@cnbc.com"}]

HTML
---
HTML dosyaları zaten düz metinden oluştuğu için onları okumakta teknik bir zorluk yok. HTML işlemede asıl istenen şey _parsing_, yani çeşitli HTML elemanlarına erişebilmektir. Söz gelişi, bir HTML'deki bağlantıları almak, tablo veriyi okumak gibi işlemler yapmamız gerekebilir. Bu tür işlemler bu yazı dizisinin kapsamını aşıyor. İlgilenenler, en çok kullanılan araçlardan biri olan [_Beautiful Soup_ modülüyle](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) denemeler yapabilirler.

Excel
---
MS Excel dosyalarını CSV olarak kaydederseniz _CSV okuma/yazma_ kısmındaki yöntemleri kullanabilirsiniz. Bunu istemiyorsanız, veya tablodaki verileriniz bu işleme uygun değilse, veya birden fazla yapraktan oluşuyorsa, Excel dosyasını doğrudan açmak için [_xlrd_ modülünü](http://xlrd.readthedocs.io/en/latest/) kullanabilirsiniz.

Daha geniş özelliklere sahip bir paket olan [_pyexcel_](http://pyexcel.readthedocs.io/en/latest/) ile hem okuma hem de yazmayı daha kolay işlemlerle yapabilirsiniz.

_Pandas_ paketinin `read_excel()` fonksiyonu ile bir Excel dosyasını doğrudan okuyarak bir veri çerçevesi haline getirebilirsiniz. Kullanımına bir örnek görmek için ["Groupby" yazımıza](http://www.veridefteri.com/2017/10/30/groupby/) bakabilirsiniz. 
Word
---
MS Word belgelerinin içinden metin almak, belge yaratmak ve varolan belgeye içerik eklemek için [_python-docx_](https://python-docx.readthedocs.io/en/latest/index.html) modülü kullanılabilir. Örnekler için Al Sweigart'ın _Automate The Boring Stuff with Python_ [kitabına](https://automatetheboringstuff.com/chapter13/) bakabilirsiniz.

PDF
---
PDF dosyalarından bilgi almak, PDF dosyası yaratmak, mevcut dosyada değişiklik yapmak gibi işler için [_pyPDF2_](https://pythonhosted.org/PyPDF2/) modülünü kullanabilirsiniz. Sweigart'ın _Automate The Boring Stuff with Python_ [kitabında] (https://automatetheboringstuff.com/chapter13/) bu modülün kullanımına dair açıklayıcı örnekler bulabilirsiniz.
ZIP
---
Python standart kütüphanesindeki [_zipfile_ modülü](https://docs.python.org/3/library/zipfile.html), ZIP formatında sıkıştırma ve açma fonksiyonları sağlar.
Basit bir örnek olarak, yukarıda kullandığımız _deneme.txt_ ve *yalanci_veri.json* dosyalarını sıkıştırarak bir arşiv dosyası oluşturalım.

In [29]:
import zipfile
with zipfile.ZipFile("arsiv.zip","w") as z:
    z.write("deneme.txt")
    z.write("yalanci_veri.json")

Mevcut bir arşive bir dosya eklemek istiyorsak, `ZipFile()` fonksiyonunun açılma modunu `"w"` yerine `"a"` yaparız.

`ZipFile()` fonksiyonu, yukarıda gördüğümüz `open()` gibi çalışır. Arşivi açmak için `ZipFile()` fonksiyonunu okuma modunda kullanırız, ve arşiv dosyası nesnesine ait `open()` metoduyla dosyayı açarız. Açılan dosyanın içeriği `read()`, `readline()` veya `readlines()` metodlarıyla okunabilir. Bu metodlar kodlanmış dizeler döndürdüğü için `decode()` dize metoduyla Unicode'a çevrilmelidir.

In [30]:
with zipfile.ZipFile('arsiv.zip') as z:
    with z.open('deneme.txt') as f:
        print(f.read().decode("utf-8"))

Ey Türk Gençliği!
Birinci vazifen, 
Türk istiklâlini, Türk Cumhuriyeti'ni, 
ilelebet muhafaza ve müdafaa etmektir.


Bir ZIP arşivindeki bir dosyayı açarak diske kaydetmek için `extract()`, arşivdeki bütün dosyaları açmak için `extractall()` fonksiyonları kullanabiliriz.

Değişkenlerimizi kaydetmek: pickle
====
Bir oturumda hazırladığınız değişkenleri, oturumu kapattığınızda kaybetmemek isterseniz bunları ikili (binary) bir veri yapısı haline getirip diske kaydetmeniz gerekir. Bu işi standart kütüphanedeki [_pickle_ modülü](https://docs.python.org/3/library/pickle.html) ile yapabilirsiniz. Neredeyse bütün Python nesnelerini (kendi tanımladığımız nesne sınıfları dahil) dosyaya kaydetmeniz ve sonra tekrar okumanız mümkündür. Bu işleme _serialization_ adı verilir.

Birkaç değişken tanımlayalım.

In [34]:
x = 3.14159
L = [1,3,2,5,4]
D = {"abc": 123, "def": 456}
def fon(x):
    return x*x

Verileri kaydetmek istediğimiz dosyayı ikili yazma modunda açalım ve _pickle_ modülündeki `dump()` fonksiyonuyla değişkenleri dosyaya ekleyelim.

In [35]:
import pickle
with open("data.p", "wb") as f:
    pickle.dump(x,f)
    pickle.dump(L,f)
    pickle.dump(D,f)
    pickle.dump(fon,f)

Değişkenleri dosyadan okumak için `load()` fonksiyonunu kullanırız. Nesneler dosyaya kondukları sırayla geri alınırlar. Değişkenlerin orijinal adını kullanmamız gerekmez.

In [36]:
with open("data.p", "rb") as f:
    y = pickle.load(f)
    print(y)
    J = pickle.load(f)
    print(J)
    F = pickle.load(f)
    print(F)
    G = pickle.load(f)
    print(G(y))

3.14159
[1, 3, 2, 5, 4]
{'abc': 123, 'def': 456}
9.869587728099999


Bazı nesneler _pickle_ ile kaydedilemezler; ağ bağlantıları, veri tabanı bağlantıları, açık dosya nesneleri gibi.

Dikkat: _pickle_ işlemleri herhangi bir emniyet tedbiri içermez. Pickle dosyasının içindeki nesne `load()` ile doğrudan doğruya çalıştırılır. Bu nesnenin içinde kötü amaçlı bir kod parçası varsa sisteminize zarar gelebilir. Bu yüzden bilmediğiniz bir yerden gelen pickle dosyalarını açmayın. 

Her programlama dilinde olduğu gibi Python’da da tekrar tekrar kullanılabilen fonksiyon ve sınıfların bir kütüphane şeklinde ayrı dosyalarda saklanması ve yeni yazılan programlara entegre edilmesi için bir mekanizma vardır. Standart kütüphaneler, SciPy ve benzeri paketler, veya kendi kişisel fonksiyon kütüphaneniz bu modül sistemiyle inşa edilir.



Modüller
====

Python’da bir modül yaratmak için özel bir işleme gerek yoktur. Python kodu içeren, `.py` uzantılı herhangi bir dosya bir modül olabilir. Sözgelişi, aşağıdaki kodu `basitmodul.py` isimli bir dosyaya yazıp kaydettiğinizde, bir modül yaratmış olursunuz.

(Burada dosyayı yaratmak için IPython'un [hücre sihirlerinden birini](http://www.veridefteri.com/2017/11/28/ipython-sihirli-ifadeler/) kullanıyoruz. İsterseniz `%%writefile ...` satırından sonraki kısmı koyalayıp bir metin editörüne yapıştırarak `basitmodul.py` adıyla kaydedebilirsiniz.)

In [1]:
%%writefile basitmodul.py
print("basitmodul çalıştırıldı.")
x = 5
def f(x):
    return x**2
def g():
    print("Merhaba")

Writing basitmodul.py


Komut satırında veya başka bir program içinde `import basitmodul` komutunu vererek (sonunda .py olmayacak) bu modülün içindeki bütün komutların işletilmesini sağlarsınız.

In [2]:
import basitmodul

basitmodul çalıştırıldı.


Elbette `basitmodul.py` dosyasını nereye kaydettiğiniz önemli. Bu örnekte, dosyanın mevcut çalışma dizininde bulunduğunu varsaydım. Bir `import` komutunda yorumlayıcı önce çalışma dizinine, sonra `PYTHONPATH` kabuk değişkeninde yazan dizinlere, sonra da kurulum sırasında belirlenmiş dizinlere bakar. İkincisini Linux bash kabuğunda `echo $PYTHONPATH` komutu ile görebilirsiniz.

Diyelim yazdığınız modülleri `modullerim` isimli bir dizin altında tutmak istiyorsunuz. Python yorumlayıcısı içindeki `sys.path` değişkeni bakılacak dizinlerin listesini tutar (önce import sys yazmayı unutmayın). Bu listeye `sys.path.append("<ev dizininiz>/modullerim")` komutuyla bir ekleme yaparak, ev dizininizin altındaki `modullerim` dizinine bakmasını sağlayabilirsiniz.

Modülü `import` etmekle yeni bir _isim alanı_ yaratmış oldunuz. Bir isim alanı bir nesnedir; dolayısıyla modülde tanımlanan değişken ve fonksiyonlara erişmek için nokta notasyonu kullanılır.

In [3]:
basitmodul.x, basitmodul.f(5)

(5, 25)

In [4]:
basitmodul.g()

Merhaba


Modülden sadece belli isimleri almak istiyorsak `from ... import` komutunu kullanabiliriz. O zaman bu değişkenler ana isim alanına aktarılmış olurlar ve onlara nokta işlemi olmadan doğrudan erişebiliriz.

In [5]:
from basitmodul import x,f
basitmodul.x, basitmodul.f(5)

(5, 25)

Yukarıdaki işlemde `g` fonksiyonunu ana isim alanına aktarmadığımız için doğrudan kullanmaya kalktığımızda hata mesajı alırız.

In [6]:
g()

NameError: name 'g' is not defined

Modüldeki bütün isimleri ana isim alanına aktarmak için `from basitmodul import *` komutu verilebilir. Ancak, böyle yaptığınızda modüldeki isimler daha önce tanımlanmış isimlerin yerine geçebilir. Söz gelişi, yeni bir `g()` fonksiyonu tanımlamış olalım.

In [7]:
def g(): print("Hasta la vista")
g()

Hasta la vista


Bu tanımdan sonra, `basitmodul`'deki bütün isimleri aşağıdaki komutla alırsak, bu fonksiyon yerine `basitmodul`'deki `g()` geçecektir.

In [8]:
from basitmodul import *
g()

Merhaba


Bu tür çatışmalara meydan vermemek için, modülleri `import *` ile yüklemek tavsiye edilmez. Modüldeki değişkenlere modül ismi aracılığıyla (meselâ `basitmodul.g()` ile) ulaşmak daha emniyetlidir.

Ancak her seferinde modül ismini  uzun uzun yazmak epeyce zahmetli olabilir ve kodun okunaklılığını azaltır. Modüle daha kısa bir isim atamak için `import ... as` komutunu kullanabiliriz.

In [9]:
import basitmodul as bm

In [10]:
bm.x, bm.f(3)

(5, 9)

`from ... import ... as` komutuyla modülden belli bir nesne (değişken, fonksiyon, vs.) alabilir ve onu yeni bir isimle kullanabilirsiniz.

In [11]:
from basitmodul import g as selamlama
selamlama()

Merhaba


## dir() fonksiyonu

Bir modülde tanımlanmış isimlerin (değişken ve fonksiyon) tam listesini görmek isterseniz `dir()` fonksiyonunu kullanabilirsiniz.

In [12]:
dir(bm)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'f',
 'g',
 'x']

`dir()` fonksiyonu sadece modüllerde değil, bir sınıf (class) içindeki isimleri almak için de kullanılabilir. Örneğin, bir liste nesnesindeki metodları listelemek için `dir(list)` komutu verebiliriz.

In [13]:
dir(list)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

Modülleri tekrar yüklemek
-----
Bir `import` işlemi bir modül dosyasını baştan sona bir kere işletir, ama ikinci bir `import` komutu dosyayı baştan çalıştırmaz. Dolayısıyla, etkileşimli çalışırken (sözgelişi Spyder gibi bir IDE ile veya Jupyter defteri ile) modül dosyasında bir değişiklik yaptıysanız, tekrar `import` yapmak bu değişikliklerin görülmesini sağlamaz.

Söz gelişi, `basitmodul.py` dosyasında `g()` fonksiyonunun tanımını değiştirelim:

In [14]:
%%writefile basitmodul.py
print("basitmodul çalıştırıldı.")
x = 5
def f(x):
    return x**2
def g():
    print("Namaste")

Overwriting basitmodul.py


Bu yeni modül dosyasını `import` ile işletmeye çalışalım:

In [15]:
import basitmodul
basitmodul.g()

Merhaba


Görüldüğü gibi `g()`'nin eski tanımını kullanıyor. Zaten başta `"basitmodul çalıştırıldı"` mesajının çıkmaması da modülün işletilmediğine işaret ediyor.

Modülünüzün güncellenmiş olarak yeniden işletilmesini istiyorsanız ya Python yorumlayıcınızı kapatıp açmalısınız (Jupyter'de kernel restart), ya da `importlib.reload()` fonksiyonunu kullanmalısınız. Bu fonksiyon bir _modül nesnesi_ döndürür.

In [16]:
from importlib import reload
bm = reload(basitmodul)

basitmodul çalıştırıldı.


In [17]:
bm.g()

Namaste


Tabii bir modül aslında dinamik bir nesne olduğu için, böyle bir değişiklik yapmak için her zaman dosyayı değiştirip tekrar yüklemek gerekmez; çalışma sırasında bir komutla da değişiklik yapılabilir. 

In [18]:
bm.g = lambda: print("Guten Tag")

In [19]:
bm.g()

Guten Tag


Paketler
=====
Bir _modül_ belli bir işe dair fonksiyonların ve sınıfların tanımlandığı bir dosyadır. İşlev olarak ilişkili, ama birbirinden ayrı birkaç modülünüz varsa bunları ortak bir dizinde tutmak mantıklı olur. Bunlar bir Python _paketi_ oluşturur.

Python'da bir _paket_ bir dizindir; bu dizinde modüller ve `__init__.py` isimli bir dosya mevcut olmalıdır. Paket yüklenirken `__init__.py` dosyasının içindeki komutlar çalıştırılır. Paket yüklemesinde ilk olarak yapılmasını istediğimiz işlemleri bu dosyaya koyabiliriz. Paket dizininde mutlaka `__init__.py` isimli bir dosya bulunmalıdır; bu boş bir dosya olabilir.

Örnek olarak, çalıştığımız dizinin altında *modullerim* isimli bir dizin, ve içinde *modul_A.py*, *modul_B.py* ve *\_\_init\_\_.py* isimli üç tane dosya yaratalım.

    modullerim/
        __init__.py
        modul_A.py
        modul_B.py

In [20]:
%%writefile modullerim/__init__.py
print("modullerim yüklendi")

Writing modullerim/__init__.py


In [21]:
%%writefile modullerim/modul_A.py    
print("Modül A yüklendi")
pi = 3.14159
def çevre(yarıçap):
    return 2*pi*yarıçap

Writing modullerim/modul_A.py


In [22]:
%%writefile modullerim/modul_B.py
print("Modül B yüklendi")
def fib(n):
    """Fibonacci dizisinin n terimi."""
    a,b=1,1
    L = [1,1]
    for i in range(n-2):
        a,b = b,a+b
        L += [b]
    return L

Writing modullerim/modul_B.py


In [23]:
import modullerim.modul_A

modullerim yüklendi
Modül A yüklendi


In [24]:
modullerim.modul_A.pi

3.14159

`from ... import` komutu ile doğrudan `modul_A` isim alanını yaratabiliriz.

In [25]:
from modullerim import modul_A
modul_A.pi, modul_A.çevre(2)

(3.14159, 12.56636)

Bu ikinci import ile `"Modul A yüklendi"` mesajının çıkmadığına dikkat edin. Bir oturumda modül içeriği sadece ilk import komutunda çalıştırılır. Sonraki import işlemlerinde modül tekrar çalıştırılmaz; yine de yukarıda görüldüğü gibi bu modülle yeni bir isim alanı oluşturmak mümkündür.

Yazma zahmetini kısaltmak için `as` kelimesiyle modüle daha kısa bir isim alanı adı verebiliriz.

In [26]:
from modullerim import modul_A as A
A.pi, A.çevre(5)

(3.14159, 31.4159)

*modul_B* içindeki bütün değişken isimlerini mevcut isim alanına ekleyebiliriz. Ama yukarıda açıkladığımız gibi bu pek tavsiye edilmez.

In [27]:
from modullerim.modul_B import *

Modül B yüklendi


In [28]:
fib(10)

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]

Yukarıdaki örneklerde `çevre()` fonksiyonunu bir modülden, `fib()` fonksiyonunu başka bir modülden aldık; bunun için de modülleri ayrıca import etme gibi bir adım atmamız icap etti. Bunun yerine bu isimleri doğrudan *\_\_init\_\_.py* içinde import edersek, modülleri ayrıca yüklememize gerek kalmaz. Bunun için *\_\_init\_\_.py* dosyasını aşağıdaki gibi değiştirelim:

In [29]:
%%writefile modullerim/__init__.py
from .modul_A import çevre
from .modul_B import fib

Overwriting modullerim/__init__.py


Modül adının başındaki noktalar, dosyanın mevcut dizinde bulunduğunu gösterir. Değişikliğin görülebilmesi için yorumlayıcıyı tekrar başlatmak (Jupyter'de "Restart Kernel"), veya yukarıda gördüğümüz gibi `reload()` kullanmak gerekir.

In [30]:
from importlib import reload
m = reload(modullerim)

Şimdi `çevre()` ve `fib()` fonksiyonlarını, tanımlandıkları modülleri import etmeye gerek kalmadan doğrudan doğruya kullanabiliriz.

In [31]:
print(m.çevre(5))
print(m.fib(10))

31.4159
[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]


## Paketlerde alt dizinler
Bir paketin altında başka dizinler de bulunabilir. Bu durumda her bir dizinin altında kendi *\_\_init\_\_.py* dosyası bulunmalıdır.

İç içe dizinler şeklinde düzenlemiş bir pakete örnek olarak _SciPy_ paketini ele alalım. Bu paketin altındaki dizinlerin bir kısmı şöyledir:

    scipy/
        integrate/
            __init__.py
            ...
        linalg/
            __init__.py
            ...
        stats/
            __init__.py
            distributions.py
            ...

Alt paketlerdeki fonksiyonları import ederken nokta (.) işlemini kullanabiliriz.

In [32]:
from scipy.integrate import quad
quad(lambda x: x**3, 1, 2)  # x^3 fonksiyonunun 1'den 2'ye kadar integrali

(3.7500000000000004, 4.1633363423443377e-14)

In [33]:
import scipy
scipy.integrate.quad(lambda x: x**3, 1, 2)

(3.7500000000000004, 4.1633363423443377e-14)

In [34]:
import scipy.integrate as intg
intg.quad(lambda x: x**3, 1, 2 )

(3.7500000000000004, 4.1633363423443377e-14)

In [35]:
from scipy.stats.distributions import norm
norm.pdf((-2,-1,0, 1, 2))  # Normal dağılımın -2, -1, 0, 1, 2 için değerleri

array([0.05399097, 0.24197072, 0.39894228, 0.24197072, 0.05399097])

Hata yakalama (exception handling) beklenmedik durumlarda programınızın bir hata mesajı vermesi ve çalışmayı durdurması yerine, hataya kendi istediğimiz şekilde cevap vermesini sağlamanın bir yoludur. Hata yakalama Python programcılığının önemli bir parçasıdır, kaynak kodunu çok karışık hale getirmeden programınızın güvenilir bir şekilde çalışmasını sağlar.


## Hatalı girdiyi yakalamak
Bir örnekle başlayalım: Etkileşimli çalışarak kullanıcıdan sayılar alan ve aldığı sayıların karesini ekrana basan bir program yazalım. Boş satır okuduğunda program sonlansın.

In [ ]:
while True:
    x = input("Bir sayı girin: ")
    if not x:
        break
    print(float(x)**2)

Örnek olarak, programımız şöyle çalışabilir.

    Bir sayı girin: 1
    1.0
    Bir sayı girin: -45.5
    2070.25
    Bir sayı girin: abc
        ---------------------------------------------------------------------------
    ValueError                                Traceback (most recent call last)
    <ipython-input-1-297c961843d7> in <module>()
          3     if not x:
          4         break
    ----> 5     print(float(x)**2)

    ValueError: could not convert string to float: 'abc'

Son girdimiz `"abc"` sayıya dönüştürülemediği için `float()` fonksiyonu bir `ValueError` hatası (Python terimiyle "exception") verdi. Böyle hatalar programımızın çalışmasını durdurur. Oysa, bir hata yakalama (exception handling) yapısı kullanırsak bu tür sorunları programımızı durdurmadan halletmemiz mümkün olur. Söz gelişi:

In [ ]:
while True:
    x = input("Bir sayı girin: ")
    if not x:
        break
    try:
        y = float(x)
    except ValueError:
        print("Geçersiz sayı")
        continue
    print(y**2)

Bu program hatalı girdi verdiğimizde ekrana bir uyarı yazar ve tekrar girdi alır:

    Bir sayı girin: 3
    9.0
    Bir sayı girin: -2
    4.0
    Bir sayı girin: abc
    Geçersiz sayı
    Bir sayı girin: 1.5
    2.25
    Bir sayı girin: 

Bu programda, hata mesajı çıkarabilecek bölümü `try:` blokunun içine aldık. Eğer `float(x)` işlemi `valueError` hatası verirse `except ValueError` bloku çalıştırılır, ve kullanıcıya bir uyarı verilerek tekrar döngünün başına dönülür. Bu sayede program durmadan hatayı yakalayıp sorunu gidermiş oluruz.

## Hata tipleri
Yukarıdaki örnekte `ValueError` hatasını yakaladık, ama başka durumlardaki hata isimlerini nereden bileceğiz? 

Öncelikle, yazdığınız her kod satırında neler olabileceğini düşünün. Hata durumu (exception) yaratan bir çok durum olabilir: Çağırdığınız fonksiyonda bir sayıyı sıfıra bölüyor olmanız mümkün mü? Bir matematiksel fonksiyona verdiğiniz değişken sayısal olmazsa ne olur? Üçüncü elemanını almaya çalıştığınız listede sadece iki eleman varsa? Açmak istediğiniz dosya diskte mevcut değilse?

Bu hata durumlarının ne olduğunu anlamak için komutları çalıştırıp ne tip hata aldığınıza bakabilirsiniz ve sonra buna göre _try/except_ blokları yazabilirsiniz. Yardım belgelerinde de fonksiyonun hangi durumlarda hangi hataları yayınlayacağına dair bilgi mevcuttur.

**Çalışma.** Yukarıdaki hata durumlarını yaratan Python kodları yazın ve hangi hataların yayınlandığına bakın. Bu hataları bir _try/except_ yapısı içine koyup uygun bulduğunuz şekilde düzenleyin.

**Çalışma.** `open()` fonksiyonunun yardım belgelerine bakarak hangi durumda hangi hataların yayınlandığını inceleyin.

Python dilindeki ön tanımlı hataların tam listesini ve hangi durumlarda yayınlandıklarını [resmi Python dökümanlarından](https://docs.python.org/3/library/exceptions.html) okuyabilirsiniz.

## try & except & else & finally

In [70]:
while True:
    try:
        benimInt = int(input("Numaranızı giriniz: "))
    except:
        print("Lütfen gerçekten numara giriniz")
        continue
    else:
        print("Teşekkürler")
        break
    finally:
        print("finally çağırıldı")

Numaranızı giriniz: enes
Lütfen gerçekten numara giriniz
finally çağırıldı
Numaranızı giriniz: deneme
Lütfen gerçekten numara giriniz
finally çağırıldı
Numaranızı giriniz: 10
Teşekkürler
finally çağırıldı


## Birden fazla hata durumu

Yukarıdaki örneğimizde, `float()` fonksiyonuna yanlış parametre vermekle ortaya çıkan `ValueError` hatasını yakalamıştık. Alternatif olarak şu kodu da kullanabilirdik:

In [ ]:
while True:
    x = input("Bir sayı girin: ")
    if not x:
        break
    try:
        y = 1/float(x)
    except:
        print("Geçersiz sayı")
        continue
    print(y)

    Bir sayı girin: 0
    Geçersiz sayı
    Bir sayı girin: abc
    Geçersiz sayı
    Bir sayı girin: 2
    0.5
    Bir sayı girin: 

Bu değişiklikle `try` bloku içindeki _herhangi_ bir hata ile `except` bloku çalıştırılır. Ancak, bu yaklaşımda farklı hataların hepsi aynı `except` blokuna yönlendirilir. Söz gelişi, yukarıda girdi olarak 0 verdiğimizde de ekrana `"Geçersiz sayı"` yazılır. Oysa bu iki ayrı hata durumunun ayrı şekilde düzenlenmesini isteyebiliriz. O zaman iki farklı `except` bloku kullanırız:

In [ ]:
while True:
    x = input("Bir sayı girin: ")
    if not x:
        break
    try:
        y = 1/float(x)
    except ValueError:
        print("Geçersiz sayı")
        continue
    except ZeroDivisionError:
        print("Sıfıra bölme")
        continue
    print(y)

Bu program farklı hatalar için farklı uyarılar gösterir:

    Bir sayı girin: abc
    Geçersiz sayı
    Bir sayı girin: 0
    Sıfıra bölme
    Bir sayı girin: 4
    0.25
    Bir sayı girin: 

## Hata durumu hiyerarşisi

Hata durumları bir nesne hiyerarşisi içinde tanımlanır. Bunların en geneli `BaseException` sınıfıdır; diğer daha özelleşmiş hata durumları bunlardan türetilir.

Hata durumları hiyerarşisinin bir bölümü şöyledir (tam bir listeyi [Python belgelerinde](https://docs.python.org/3/library/exceptions.html#exception-hierarchy) bulabilirsiniz):

    BaseException
     +-- SystemExit
     +-- KeyboardInterrupt
     +-- Exception
          +-- StopIteration
          +-- StopAsyncIteration
          +-- ArithmeticError
          |    +-- FloatingPointError
          |    +-- OverflowError
          |    +-- ZeroDivisionError
          +-- ImportError
          |    +-- ModuleNotFoundError
          +-- LookupError
          |    +-- IndexError
          |    +-- KeyError
          +-- OSError
          +-- ValueError

Bu hiyerarşi sebebiyle, alt seviye bir hatayı yayınlayan bir kod, onun üstündeki hataları da yayınlar. Söz gelişi, `1/0` işlemi `ZeroDivisionError`, `ArithmeticError`, `Exception` ve `BaseException` hatalarının hepsine uyar.

Bir try/except yapısında bir hata durumu belirtmezsek en genel durum olan `BaseException` yayınlanır.

In [1]:
try:
    1/0
except:
    print("Bir hata oldu.")

Bir hata oldu.


Ama böyle bir kullanım, okunaklı yazılım geliştirme açısından doğru değildir. Eğer `try` blokumuz genişse ve birden fazla farklı hata olması ihtimali varsa, hepsi birden bu mesajı verir, ve hangi hatanın gerçekleştiğini tespit etmemiz mümkün olmaz.

In [2]:
try:
    int("abc")
except:
    print("Bir hata oldu.")

Bir hata oldu.


Belirsizliği azaltmak için en iyi yol, hiyerarşide en alt noktadaki (en dar kapsamlı) hata durumunu yakalamak ve ona göre ayrı `except` blokları içinde sorunu gidermektir.

In [3]:
try:
    2.5**1000
except OverflowError:
    print("İşlem çok büyük.")
except ZeroDivisionError:
    print("Sıfıra bölme.")

İşlem çok büyük.


Hatta, yaptığınız işlemin yeni bir hata durumu olmasını da sağlayabilirsiniz. Öntanımlı hata durumlarından yeni hatalar türetmeyi aşağıda göreceğiz.

## Fonksiyonlarımızda hata durumu yayınlamak

Gördüğümüz gibi birçok Python fonksiyonu normal işleyişe uymayan durumlarda bir hata durumu yayınlıyor, ve programımızda bu hata durumunu yakalayarak işlem yapıyoruz. Kendi yazdığımız fonksiyonların içinde `raise` komutu kullanarak bir hata durumu yayınlanmasını sağlayabiliriz. Örnek olarak, negatif argüman aldığında `ValueError` yayınlayan bir faktöriyel fonksiyonu yazalım. Hata mesajını değiştirmemiz de mümkündür:

In [4]:
def faktöryel(x):
    x = int(x)    
    if x<0:
        raise ValueError("Negatif değer")
    p = 1
    for i in range(1,x+1):
        p *= i
    return p

Şimdi bu fonksiyonu bir try/except bloku içinde kullanalım.

In [5]:
for x in [5, -5, "abc", 5]:
    try:
        y = faktöryel(x)
    except ValueError as e:
        print(x,": ", e)
        continue
    print(y)

120
-5 :  Negatif değer
abc :  invalid literal for int() with base 10: 'abc'
120


Bu koddaki `except ValueError as e:` komutu ile hata durumu `e` isimli bir yerel değişkende saklanabilir ve blok içinde kullanılabilir. Yukarıdaki gibi `print()` içinde kullanıldığında hata mesajını ekrana basarız. Negatif girdi ve harf girdisi durumlarında farklı hata mesajları çıktığına dikkat edin.

## Yeni hata durumları yaratmak

Python'un standart hata durumlarına ek olarak, kendi hata durumlarımızı da yaratabiliriz. Yukarıda gördüğümüz hata durumu hiyerarşisi, aslında bir nesne hiyerarşisidir. Nesne sınıfları tanımlamayı sonraki bölümlerde göreceğiz, ama buradaki örneği nesne programlama bilmeden de uygulayabilirsiniz.

Yeni bir hata tanımlarken varolan bir hatayı temel alırız. Söz gelişi, genel `Exception` nesne sınıfından türetilmiş bir `VektörBoyuHatası` tanımlayalım. 

In [6]:
class VektörBoyuHatası(Exception):
    pass

Buradaki `pass`  kelimesi etkisiz bir komuttur. Python sözdizimi gereğince doldurulması gereken bir yere herhangi bir kod koymak istemediğimizde kullanırız.

Şimdi iki sayı listesinin iç çarpımını veren bir fonksiyon yazalım. Listeler aynı uzunlukta değilse iç çarpım tanımlı olmaz; bu durumda `VektörBoyuHatası` yayınlayalım.

In [7]:
def iç_çarpım(L1, L2):
    if len(L1)!=len(L2):
        raise VektörBoyuHatası("Parametreler aynı sayıda elemandan oluşmalı.")
    return sum( [a*b for (a,b) in zip(L1,L2)] )

In [8]:
iç_çarpım([1,2,3], [-1,0,1])

2

In [9]:
iç_çarpım([1,2,3,4], [-1,0,1])

VektörBoyuHatası: Parametreler aynı sayıda elemandan oluşmalı.

Bu fonksiyonu bir try/except yapısı içinde kullanabiliriz:

In [10]:
try:
    iç_çarpım([1,2,3,4], [-1,0,1])
except VektörBoyuHatası as e:
    print(e)

Parametreler aynı sayıda elemandan oluşmalı.


Hata yakalama (exception handling) beklenmedik durumlarda programınızın bir hata mesajı vermesi ve çalışmayı durdurması yerine, hataya kendi istediğimiz şekilde cevap vermesini sağlamanın bir yoludur. Hata yakalama Python programcılığının önemli bir parçasıdır, kaynak kodunu çok karışık hale getirmeden programınızın güvenilir bir şekilde çalışmasını sağlar.
